In [1]:
import numpy as np
import torch

from torch.utils.data import Dataset, DataLoader


1. generate points. Just a line so a vector with 10 points with varying starting points that is sorted. return the first point. It is vector, label

Encapsulate that in a Dataloader object.


In [2]:
amount_of_points = 10
min_val = 0
max_val = 30

def get_line():
    line_points = np.random.randint(low=min_val, high=max_val, size=(amount_of_points,))
    line_points.sort()
    torch_line_points = torch.from_numpy(line_points)
    return torch_line_points, torch_line_points[0]

print(get_line())
print(type(get_line()[1]))

class CustomDataset(Dataset):
    def __init__(self, size):
        self.size = size
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        input_data, label = get_line()
        return input_data, label
    
dataset_size = 1000
batch_size = 16


train_dataset = CustomDataset(dataset_size)
test_dataset = CustomDataset(dataset_size//10)

# No need for shuffle
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


(tensor([ 0,  0,  1, 12, 13, 22, 22, 25, 25, 26]), tensor(0))
<class 'torch.Tensor'>


2. Embedding layer that takes the points and embedds them into a different space.

In [3]:
# TODO what about the embedding dim
embedding_dim = 64
embedding_layer = torch.nn.Embedding(num_embeddings=max_val + 1, embedding_dim=embedding_dim)

3. Positional encoding: ignore for now

In [4]:
# ignored

4. Attention Layer. Also a network/Layer

In [5]:
from torch import nn
import torch
import torch.nn.functional as F

class AttentionLayer(nn.Module):
    def __init__(self, feature_size):
        super(AttentionLayer, self).__init__()
        self.feature_size = feature_size

        # calculate Q, K, V from the same source
        self.k = nn.Linear(feature_size, feature_size)
        self.q = nn.Linear(feature_size, feature_size)
        self.v = nn.Linear(feature_size, feature_size)

    def forward(self, x, mask=None):
        # linear trans
        keys = self.k(x)
        queries = self.q(x)
        values = self.v(x)

        # move into init?
        # Why the f do I need the first parameter as a tensor? Wrong function call? how about np.sqrt?
        scaling_factor = torch.sqrt(torch.tensor(self.feature_size, dtype=torch.float32))
        
        scores = torch.matmul(queries, keys.transpose(-2, -1)) / scaling_factor

        # optional: add mask here

        # appy softmax, convert scores to floating point otherwise the softmax doesn't work
        # TODO figure out why
        attention_weights = F.softmax(scores.float(), dim=1)

        output = torch.matmul(attention_weights, values)

        return output, attention_weights

Final network, with all layers + a linear layer (should suffice)

In [6]:
class Transfomer(nn.Module):
    def __init__(self):
        super(Transfomer, self).__init__()
        self.embedding = torch.nn.Embedding(num_embeddings=max_val + 1, embedding_dim=embedding_dim)
        self.attention_layer = AttentionLayer(feature_size=embedding_dim)
        self.lin_net = nn.Linear(embedding_dim, out_features=max_val + 1)

    def forward(self, x):
        emb_x = self.embedding(x)
        # ignoring the attention weights
        att_x, _ = self.attention_layer(emb_x)
        out_x = torch.amax(self.lin_net(att_x), dim=-2)
        return out_x

5. Training loop

In [7]:
def onehotencodevalue(size:int, label:int):
    vector = torch.zeros(size)
    vector[label] = 1.0

    return vector

def onehotencodebatchvalues(size:int, labels):
    oneHotencodedBatch = torch.zeros((len(labels), size))

    for idx, label in enumerate(labels):
        OHE_vector = onehotencodevalue(size, label)
        oneHotencodedBatch[idx] = OHE_vector
    
    return oneHotencodedBatch

OHEBathc = onehotencodebatchvalues(max_val + 1, [4, 7, 9, 10])
OHEBathc.shape

torch.Size([4, 31])

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and  dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch_number, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        # take the highest predicted class.
        # TODO THIS SEEMS WRONG; TAKING THE HIGHES PREDICTED CLASS WILL STIFEN LEARNING NO?
        pred = model(X)

        target_vector = onehotencodebatchvalues(size=max_val+1, labels=y)
        loss = loss_fn(pred.float(), y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_number % 100 == 0:
            loss, current = loss.item(), (batch_number + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

6. Testing loop

In [9]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            target_vector = onehotencodebatchvalues(size=max_val+1, labels=y)
            test_loss += loss_fn(pred.float(), y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

7. Execution 

In [10]:
model = Transfomer()
learning_rate = 0.1

# defines the "correct" the output is
loss_fn = nn.CrossEntropyLoss()
# defines how to adapt the model parameters depending on the (input, loss)
# here we use stochastic gradient descent
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
	# testing for each epoch to track the models performance during training.
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3.395942  [   16/ 1000]
Test Error: 
 Accuracy: 81.0%, Avg loss: 1.028517 

Epoch 2
-------------------------------
loss: 0.780109  [   16/ 1000]
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.574464 

Epoch 3
-------------------------------
loss: 0.477492  [   16/ 1000]
Test Error: 
 Accuracy: 86.0%, Avg loss: 0.509828 

Epoch 4
-------------------------------
loss: 0.146055  [   16/ 1000]
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.256370 

Epoch 5
-------------------------------
loss: 0.367672  [   16/ 1000]
Test Error: 
 Accuracy: 98.0%, Avg loss: 0.126452 

Epoch 6
-------------------------------
loss: 0.287313  [   16/ 1000]
Test Error: 
 Accuracy: 92.0%, Avg loss: 0.233219 

Epoch 7
-------------------------------
loss: 0.135310  [   16/ 1000]
Test Error: 
 Accuracy: 93.0%, Avg loss: 0.270794 

Epoch 8
-------------------------------
loss: 0.328723  [   16/ 1000]
Test Error: 
 Accuracy: 96.0%, Avg loss: 0.201279 

Epoch 9
----------------

In [11]:
x,y = get_line()
model_output = model(x).argmax()

print("model input: ", x)
print("expected output: first element: ", y.item())
print("actual output: ", model_output.item())

if model_output == y:
    print("Output was correct")
else:
    print("Output was incorrect!")

model input:  tensor([ 2,  3,  3,  4,  6, 13, 18, 19, 23, 24])
expected output: first element:  2
actual output:  2
Output was correct
